# Effect of noise on the accuracy of OneForest mapping method

We evaluate the effect of increasing the noise in ground data and drone on the accuracy of the different mapping methods to recover the true mapping bewtween the two data sources. For evaluation, we create synthetic datasets on scratch or based on real drone images from Ecuador or NEON dtasets.

We can produce noisy data in different ways:
- GPS noise in ground data as random gaussian noise.
- GPS noise in drone data as a uniform shift, since all the trees from the same drone image are identically noisy.
- Noise in inexact tree detection of DeepForest: we play on the threshold score of deepforest, defined as the score at which we accept bounding boxes.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec
import rasterio
from rasterio.plot import reshape_as_raster, reshape_as_image

import tensorflow

import matplotlib.pylab as plt
import ot
import ot.plot
import cv2

import seaborn as sns

from sklearn.metrics import accuracy_score

import matplotlib.image as mpimg
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable


Bad key text.latex.unicode in file /anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file /anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/bl

In [2]:
import sys
package = os.path.dirname(os.getcwd())
sys.path.append(package)
sys.path.append(package + '/utils')
sys.path

['',
 '/anaconda3/lib/python36.zip',
 '/anaconda3/lib/python3.6',
 '/anaconda3/lib/python3.6/lib-dynload',
 '/Users/kenzaamara/.local/lib/python3.6/site-packages',
 '/anaconda3/lib/python3.6/site-packages',
 '/anaconda3/lib/python3.6/site-packages/aeosa',
 '/Users/kenzaamara/Documents/ETH Zurich/Master/Master Thesis/code/oneforest',
 '/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/Users/kenzaamara/.ipython',
 '/Users/kenzaamara/PycharmProjects/oneforest',
 '/Users/kenzaamara/PycharmProjects/oneforest/utils']

In [3]:
import utils 

from utils.citizen_science import *
from utils.extract_features import *

from utils.deepforest_detection import *
from utils.deepforest_evaluation import *

from utils.visualisation import *
from utils.plot_folium import *
from utils.plot_density import *

from utils.mapping import *

Using TensorFlow backend.


In [4]:
def index_nn(X_ground_nn, X_drone_nn):
    
    n = len(X_ground_nn)
    m = len(X_drone_nn)
    X = np.concatenate([X_ground_nn, X_drone_nn])
    
    dists, idxs = nearest_neighbors(X, nbr_neighbors=n+m)

    ground_index = []
    for i in range(n):
        p = 1
        id = X[i,2]
        while(X[idxs[i][p], 2] == id):
            p += 1
        j = idxs[i,p]
        ground_index.append(j-n)
    ground_index = np.array(ground_index, dtype = np.int32)
    return(ground_index)

## Simulate increasing random noise in ground data

### Ecuador synthetic dataset

In [ ]:
boxes = pd.read_csv('Ecuador/evaluation/hand_test_example.csv', names = ['img_path', 'xmin', 'ymin', 'xmax', 'ymax', 'label', 'is_musacea'])
boxes

In [ ]:
boxes['x'] = (boxes.xmin + boxes.xmax)/2
boxes['y'] = (boxes.ymin + boxes.ymax)/2
boxes['img_path'] = 'Flora Pluas RGB_7_3800_7600_7800_11600.png'

In [ ]:
boxes

In [ ]:
site_data = pd.read_csv('Ecuador/features/ortho_data.csv')

rx_meters = []
ry_meters = []

for index, row in site_data.iterrows():
    height_pxl = (row.lat_max - row.lat_min)/row.ratio_y
    length_pxl = (row.lon_max - row.lon_min)/row.ratio_x
    
    coords_1 = (row.lat_min, row.lon_min)
    coords_2 = (row.lat_max, row.lon_min)

    height = geopy.distance.vincenty(coords_1, coords_2).km*1000

    coords_1 = (row.lat_min, row.lon_min)
    coords_2 = (row.lat_min, row.lon_max)

    length = geopy.distance.vincenty(coords_1, coords_2).km*1000
    
    rx_meters.append(length_pxl/length)
    ry_meters.append(height_pxl/height)
    
print(np.mean(rx_meters))
print(np.mean(ry_meters))

# 1 meter is represented by approximatively 160 pixels

In [ ]:
# Observe what is the effect of the random gaussian noise on ground data
img_path = 'Ecuador/evaluation/test_example.tif'
im = cv2.imread(img_path)
annot = box_to_annotation(boxes[['xmin', 'ymin', 'xmax', 'ymax']].to_numpy())
im = draw_annotations(im, annot, color=(0, 0, 255), label_to_name=None, plot = False, cv2_authorized = False, thickness = 4)

X_d = boxes[['x', 'y']].to_numpy()
X_g = boxes[['x', 'y']].to_numpy()

sigmas = np.array([0.5, 1.5, 3])*160


n = len(X_d)
for k in range(len(sigmas)):
    noise = np.random.normal(0, sigmas[k], (n, 2))
    X_ground = X_g + noise
    X_drone = X_d
    
    
    plt.figure(figsize=(15,15))
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))

    for i in range(n):
        plt.scatter(x=X_ground[i][0], y=X_ground[i][1], c='b', s=40)
        plt.scatter(x=X_drone[i][0], y=X_drone[i][1], c='r', s=40)

        plt.plot([X_ground[i][0], X_drone[i][0]], [X_ground[i][1], X_drone[i][1]], color='red', marker=None, linewidth=0.5, markersize=2)
    plt.show()

In [ ]:
def plot_mapping(X_drone, X_ground, idx):

    img_path = 'Ecuador/evaluation/test_example.tif'
    im = cv2.imread(img_path)
    annot = box_to_annotation(boxes[['xmin', 'ymin', 'xmax', 'ymax']].to_numpy())
    im = draw_annotations(im, annot, color=(0, 0, 255), label_to_name=None, plot = False, cv2_authorized = False, thickness = 4)

    plt.figure(figsize=(15,15))
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))

    X_drone_pred = X_drone[idx]
    print(idx)
    n = len(X_ground)
    y_true = range(n)
    res = y_true == idx
    print(res)

    for i in range(n):
        plt.scatter(x=X_ground[i][0], y=X_ground[i][1], c='b', s=40)
        plt.scatter(x=X_drone[i][0], y=X_drone[i][1], c='r', s=40)

        #plt.plot([X_ground[i][0], X_drone[i][0]], [X_ground[i][1], X_drone[i][1]], color='red', marker="o", linewidth=0.5, markersize=2)
        
        if res[i] == True:
            plt.plot([X_ground[i][0], X_drone_pred[i][0]], [X_ground[i][1], X_drone_pred[i][1]], color='green', marker=None, linewidth=1)
        if res[i] == False:
            plt.plot([X_ground[i][0], X_drone_pred[i][0]], [X_ground[i][1], X_drone_pred[i][1]], color='red', marker=None, linewidth=1)

    plt.show()

In [ ]:
def evaluation_noise(sigmas, X_d, mus_d=None, mus_g=None):
    
    Acc_nn = []
    Acc_ot_non_greedy = []
    Acc_ot_greedy = []
    Acc_ot_cnn = []
    Acc_gw = []
    

    n = len(X_d)
    for k in range(len(sigmas)):
        sigma = sigmas[k]
        noise = np.random.normal(0, sigma, (n, 2))
        X_ground = X_d + noise
        X_drone = X_d
        
        y_true = range(n)

        # Nearest Neighbours
        X_drone_nn = np.hstack((X_drone, np.zeros((n, 1), dtype=np.int32)))
        X_ground_nn = np.hstack((X_ground, np.zeros((n, 1), dtype=np.int32)+1))
        idx = index_nn(X_ground_nn, X_drone_nn)
        Acc_nn.append(accuracy_score(y_true, idx))
        #plot_mapping(X_drone, X_ground, idx)

        # OT ground to drone
        G = OT_sinkhorn(X_drone, X_ground, lambd = 0.01)
        idx = np.argmax(G, axis = 0)
        Acc_ot_non_greedy.append(accuracy_score(y_true, idx))
        #plot_mapping(X_drone, X_ground, idx)
        
        # OT greedy
        G = OT_sinkhorn(X_drone, X_ground, lambd = 0.01)
        cost = -G*10e4
        row_ind, col_ind = linear_sum_assignment(cost)
        Acc_ot_greedy.append(accuracy_score(y_true, col_ind))
        #plot_mapping_inv(X_drone, X_ground, idx)
        
        
        if mus_g is not None:
            # OT with CNN filtering ground to drone
            G = OT_scores_sinkhorn(X_drone, X_ground, mus_d, mus_g, mu = 1, lambd = 0.01)
            cost = -G
            row_ind, col_ind = linear_sum_assignment(cost)
            Acc_ot_cnn.append(accuracy_score(y_true, col_ind))
        
        
        # Gromov-Wasserstein
        G = gromov_wasserstein(X_drone, X_ground)
        idx = np.argmax(G, axis = 0)
        Acc_gw.append(accuracy_score(y_true, idx))
        
        
    return(Acc_nn, Acc_ot_non_greedy, Acc_ot_greedy, Acc_ot_cnn, Acc_gw)

In [ ]:
df = pd.read_csv('Ecuador/annotations/final_annotations.csv', index_col = 0)
df.columns

In [ ]:
df = pd.read_csv('Ecuador/evaluation/hand_test_example.csv', names = ['img_path', 'xmin', 'ymin', 'xmax', 'ymax', 'label', 'is_musacea'])
df[['x', 'y']] = df.apply(lambda x: [get_center(x.xmin,x.xmax), get_center(x.ymin,x.ymax)], axis=1, result_type="expand")

df.img_path = 'Flora Pluas RGB_7_3800_7600_7800_11600.png'

cnn_model = tensorflow.keras.models.load_model('Ecuador/cnn/cnn_model')
site_name = 'Flora Pluas RGB'
df = predict_musacea(df, site_name, cnn_model)

# True is_musacea ('is_musacea_g')

y_true = np.load('Ecuador/cnn/test/labels.npy')
y_true = y_true.astype(int)
df['is_musacea_g'] = y_true

df = df.rename(columns = {"is_musacea": "is_musacea_d"})

In [ ]:
new_df = df.copy()
n = len(new_df)
N = int(n*0.2)
Test = []
for i in range(5):
    test = new_df.sample(n=N, random_state=1)
    Test.append(test)
    new_df = new_df.drop(test.index) 

In [ ]:
sigmas = np.array([0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4])*160

def run_evaluation_test_Ecuador_tile(Test, sigmas):
    LIST_ACC_TEST = []
    res = pd.DataFrame(columns=['sigma', 'accuracy', 'method', 'test'])
    k = 0
    for df_test in Test:
        k +=1
        X_d = df_test[['x', 'y']].to_numpy()

        mus_d = df_test.is_musacea_d.to_numpy()
        mus_d = np.vstack(mus_d).reshape(-1)
        mus_g = df_test.is_musacea_g.to_numpy()

        Acc_nn, Acc_ot_non_greedy, Acc_ot_greedy, Acc_ot_cnn, Acc_gw = evaluation_noise(sigmas, X_d, mus_d, mus_g)

        ACC_TEST = [Acc_nn, Acc_ot_non_greedy, Acc_ot_greedy, Acc_ot_cnn, Acc_gw]
        methods = ['NN', 'OT non greedy', 'OT greedy', 'OT CNN', 'GW']
        
        n = len(sigmas)
        for i in range(len(methods)):
            df_sup = pd.DataFrame()
            df_sup['sigma'] = sigmas/160
            df_sup['accuracy'] = ACC_TEST[i]
            df_sup['method'] = [methods[i]]*n
            df_sup['test'] = [k]*n
            res = pd.concat([res, df_sup], ignore_index=True)
            
        #print("ACC_TEST is: ", ACC_TEST)
        #LIST_ACC_TEST.append(ACC_TEST)
    return(res)

In [ ]:
res_Ecuador = run_evaluation_test_Ecuador_tile(Test, sigmas)
res_Ecuador

In [ ]:
sns.lineplot(data=res_Ecuador, x="sigma", y="accuracy", hue="method")

In [ ]:
res_Ecuador.to_csv('Ecuador/results/evaluation_nn_ot_gw.csv')

In [5]:
res_Ecuador = pd.read_csv('Ecuador/results/evaluation_nn_ot_gw.csv')
res_Ecuador = res_Ecuador[res_Ecuador.method != 'OT non greedy']
res_Ecuador = res_Ecuador.replace({'OT greedy':'OT on GPS position', 'OT CNN':'OT on GPS position + Tree species'})

FileNotFoundError: [Errno 2] No such file or directory: 'Ecuador/results/evaluation_nn_ot_gw.csv'

In [ ]:
res_Ecuador_gmn = pd.read_csv('Ecuador/results/evaluation_gmn.csv')
res_Ecuador = pd.concat([res_Ecuador, res_Ecuador_gmn])

In [ ]:
fig, ax = plt.subplots(figsize=[15, 10])
ax.set_xlabel('sigma (m)')
ax.set_ylabel('accuracy')
ax.set_title('Evaluation of Mapping Methods when Increasing Noise in GPS coordinates of Ground Measurements\n (Synthetic Dataset based on 105 trees in Ecuador)')

sns.lineplot(data=res_Ecuador, x="sigma", y="accuracy", hue="method", ax=ax)

### NEON synthetic dataset

In [ ]:
df = pd.read_csv('NEON/data/true_matching.csv', index_col = 0)
len(df)

In [ ]:
df.columns

In [ ]:
df.scientificName.loc[0]

In [ ]:
# initialize empty map zoomed in on 
m = folium.Map(location=[30, 120], zoom_start=13,tiles='CartoDBPositron')


X_street = df[['plotLatitude', 'plotLongitude']].to_numpy()
#add markers

for i in range(len(X_street)):
    #folium.Marker(each, icon=folium.Icon(color='blue', icon='cloud')).add_to(m)
    #print(df.scientificName.loc[i], df.stemDiameter.loc[i], df.height.loc[i])
    #my_string = '<b>Species</b>: {}, <br><b>Diameter</b>: {},  <br><b>Height</b>: {}'.format(df.scientificName.loc[i], df.stemDiameter.loc[i], df.height.loc[i])  
    #folium.CircleMarker(location=X_street[i], radius=3, weight = 0, fill_color='blue', fill_opacity=0.4, popup = Popup(my_string)).add_to(m)
    folium.CircleMarker(location=X_street[i], radius=3, weight = 0, fill_color='red', fill_opacity=0.4).add_to(m)

folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
    ).add_to(m)

In [ ]:
m

In [ ]:
# Plot on one tile 'BART_036.tif'

sub = df[df.img_path == 'BART_036.tif']
X_drone = sub[['x', 'y']].to_numpy()
print(len(X_drone))
n = len(X_drone)

sigma = 30
noise = np.random.normal(0, sigma, (n, 2))
X_ground = X_drone + noise

src = rasterio.open('NEON/images/BART_036.tif')
raster = src.read()
image = reshape_as_image(raster)

plt.figure(figsize=(15,15))
plt.imshow(image.astype(int))

for i in range(len(X_drone)):
    plt.scatter(x=X_drone[i][0], y=X_drone[i][1], c='r', s=40)
    plt.scatter(x=X_ground[i][0], y=X_ground[i][1], c='b', s=40)
    plt.plot([X_ground[i][0], X_drone[i][0]], [X_ground[i][1], X_drone[i][1]], color='red', marker=None, linewidth=0.5, markersize=2)
plt.show()

In [ ]:
def evaluation_noise(sigmas, X_drone):
    
    Acc_nn = []
    Acc_ot_non_greedy = []
    Acc_ot_greedy = []
    Acc_ot_cnn = []
    Acc_gw = []
    

    n = len(X_drone)
    for k in range(len(sigmas)):
        noise = np.random.normal(0, sigmas[k], (n, 2))
        X_ground = X_drone + noise
        
        y_true = range(n)

        # Nearest Neighbours
        X_drone_nn = np.hstack((X_drone, np.zeros((n, 1), dtype=np.int32)))
        X_ground_nn = np.hstack((X_ground, np.zeros((n, 1), dtype=np.int32)+1))
        idx = index_nn(X_ground_nn, X_drone_nn)
        Acc_nn.append(accuracy_score(y_true, idx))
        #plot_mapping(X_drone, X_ground, idx)

        # OT ground to drone
        G = OT_sinkhorn(X_drone, X_ground, lambd = 0.01)
        idx = np.argmax(G, axis = 0)
        Acc_ot_non_greedy.append(accuracy_score(y_true, idx))
        #plot_mapping(X_drone, X_ground, idx)
        
        # OT greedy
        G = OT_sinkhorn(X_drone, X_ground, lambd = 0.01)
        cost = -G*10e4
        row_ind, col_ind = linear_sum_assignment(cost)
        Acc_ot_greedy.append(accuracy_score(y_true, col_ind))
        #plot_mapping_inv(X_drone, X_ground, idx)
        
        
        # Gromov-Wasserstein
        G = gromov_wasserstein(X_drone, X_ground)
        idx = np.argmax(G, axis = 0)
        Acc_gw.append(accuracy_score(y_true, idx))
        
        
    return(Acc_nn, Acc_ot_non_greedy, Acc_ot_greedy, Acc_ot_cnn, Acc_gw)

In [ ]:
sigmas = np.array([0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4])

def run_evaluation_test(Test, sigmas):
    LIST_ACC_TEST = []
    res = pd.DataFrame(columns=['sigma', 'accuracy', 'method', 'test'])
    k = 0
    for df_test in Test:
        k +=1
        Acc_nn_sites = []
        Acc_ot_non_greedy_sites = []
        Acc_ot_greedy_sites = []
        Acc_gw_sites = []
        list_site = np.unique(df_test['site'].to_numpy())
        for name in list_site:
            site = df_test[df_test.site == name]
            
            X_drone = site[['X', 'Y']].to_numpy()
            if(len(site)>1):
                Acc_nn, Acc_ot_non_greedy, Acc_ot_greedy,_, Acc_gw = evaluation_noise(sigmas, X_drone)
                Acc_nn_sites.append(Acc_nn)
                Acc_ot_non_greedy_sites.append(Acc_ot_non_greedy)
                Acc_ot_greedy_sites.append(Acc_ot_greedy)
                Acc_gw_sites.append(Acc_gw)
        
        ACC_TEST = [np.mean(Acc_nn_sites, axis = 0), np.mean(Acc_ot_non_greedy_sites, axis = 0), np.mean(Acc_ot_greedy_sites, axis = 0), np.mean(Acc_gw_sites, axis = 0)]
        methods = ['NN', 'OT non greedy', 'OT greedy', 'GW']
        n = len(sigmas)
        for i in range(len(methods)):
            df_sup = pd.DataFrame()
            df_sup['sigma'] = sigmas
            df_sup['accuracy'] = ACC_TEST[i]
            df_sup['method'] = [methods[i]]*n
            df_sup['test'] = [k]*n
            res = pd.concat([res, df_sup], ignore_index=True)
            
        #print("ACC_TEST is: ", ACC_TEST)
        #LIST_ACC_TEST.append(ACC_TEST)
    return(res)
        

In [ ]:
# Creat 5 testing folds to have a variance and a mean for the mapping accuracy
new_df = df.copy()
n = len(new_df)
N = int(n*0.2)
Test = []
for i in range(5):
    test = new_df.sample(n=N, random_state=1)
    Test.append(test)
    new_df = new_df.drop(test.index) 

In [ ]:
res_NEON = run_evaluation_test(Test, sigmas)
res_NEON

In [ ]:
res_NEON.to_csv('NEON/results/evaluation_nn_ot_gw.csv')

In [ ]:
res_NEON = pd.read_csv('NEON/results/evaluation_nn_ot_gw.csv')
res_NEON = res_NEON[res_NEON.method != 'OT non greedy']
res_NEON = res_NEON.replace({'OT greedy':'OT'})

In [ ]:
sns.lineplot(data=res_NEON, x="sigma", y="accuracy", hue="method")

In [ ]:
res_NEON_gmn = pd.read_csv('NEON/results/evaluation_gmn.csv')
res_NEON = pd.concat([res_NEON, res_NEON_gmn])

In [ ]:
fig, ax = plt.subplots(figsize=[15, 10])
ax.set_xlabel('sigma (m)')
ax.set_ylabel('accuracy')
ax.set_title('Evaluation of Mapping Methods when Increasing Noise in GPS on a Synthetic Dataset\n (3839 trees of NEON Dataset)')

sns.lineplot(data=res_NEON, x="sigma", y="accuracy", hue="method", ax=ax)

## Noise in Ground AND Drone data (GPS position) - purely synthetic dataset

We generate points in a grid. We add random gaussian noise in ground data and a shift in drone data. We observe the ability of different mapping methods to recover the true mapping.

We generate the points based on a random Poisson-disk Sampling. Similar to the PPP, the Poisson-disk Sampling generates the points randomly, but ensures that the points are at least a distance d apart from each other.

In [ ]:
# Choose up to k points around each reference point as candidates for a new
# sample point
k = 30

# Minimum distance between samples
r = 3

width, height = 60, 45 # --> 200 points approximately 
#width, height = 25, 25 # --> 50 points approximately 

# Cell side length
a = r/np.sqrt(2)
# Number of cells in the x- and y-directions of the grid
nx, ny = int(width / a) + 1, int(height / a) + 1

# A list of coordinates in the grid of cells
coords_list = [(ix, iy) for ix in range(nx) for iy in range(ny)]
# Initilalize the dictionary of cells: each key is a cell's coordinates, the
# corresponding value is the index of that cell's point's coordinates in the
# samples list (or None if the cell is empty).
cells = {coords: None for coords in coords_list}

def get_cell_coords(pt):
    """Get the coordinates of the cell that pt = (x,y) falls in."""

    return int(pt[0] // a), int(pt[1] // a)

def get_neighbours(coords):
    """Return the indexes of points in cells neighbouring cell at coords.

    For the cell at coords = (x,y), return the indexes of points in the cells
    with neighbouring coordinates illustrated below: ie those cells that could 
    contain points closer than r.

                                     ooo
                                    ooooo
                                    ooXoo
                                    ooooo
                                     ooo

    """

    dxdy = [(-1,-2),(0,-2),(1,-2),(-2,-1),(-1,-1),(0,-1),(1,-1),(2,-1),
            (-2,0),(-1,0),(1,0),(2,0),(-2,1),(-1,1),(0,1),(1,1),(2,1),
            (-1,2),(0,2),(1,2),(0,0)]
    neighbours = []
    for dx, dy in dxdy:
        neighbour_coords = coords[0] + dx, coords[1] + dy
        if not (0 <= neighbour_coords[0] < nx and
                0 <= neighbour_coords[1] < ny):
            # We're off the grid: no neighbours here.
            continue
        neighbour_cell = cells[neighbour_coords]
        if neighbour_cell is not None:
            # This cell is occupied: store this index of the contained point.
            neighbours.append(neighbour_cell)
    return neighbours

def point_valid(pt):
    """Is pt a valid point to emit as a sample?

    It must be no closer than r from any other point: check the cells in its
    immediate neighbourhood.

    """

    cell_coords = get_cell_coords(pt)
    for idx in get_neighbours(cell_coords):
        nearby_pt = samples[idx]
        # Squared distance between or candidate point, pt, and this nearby_pt.
        distance2 = (nearby_pt[0]-pt[0])**2 + (nearby_pt[1]-pt[1])**2
        if distance2 < r**2:
            # The points are too close, so pt is not a candidate.
            return False
    # All points tested: if we're here, pt is valid
    return True

def get_point(k, refpt):
    """Try to find a candidate point relative to refpt to emit in the sample.

    We draw up to k points from the annulus of inner radius r, outer radius 2r
    around the reference point, refpt. If none of them are suitable (because
    they're too close to existing points in the sample), return False.
    Otherwise, return the pt.

    """
    i = 0
    while i < k:
        rho, theta = np.random.uniform(r, 2*r), np.random.uniform(0, 2*np.pi)
        pt = refpt[0] + rho*np.cos(theta), refpt[1] + rho*np.sin(theta)
        if not (0 <= pt[0] < width and 0 <= pt[1] < height):
            # This point falls outside the domain, so try again.
            continue
        if point_valid(pt):
            return pt
        i += 1
    # We failed to find a suitable point in the vicinity of refpt.
    return False

# Pick a random point to start with.
pt = (np.random.uniform(0, width), np.random.uniform(0, height))
samples = [pt]
# Our first sample is indexed at 0 in the samples list...
cells[get_cell_coords(pt)] = 0
# ... and it is active, in the sense that we're going to look for more points
# in its neighbourhood.
active = [0]

nsamples = 1
# As long as there are points in the active list, keep trying to find samples.
while active:
    # choose a random "reference" point from the active list.
    idx = np.random.choice(active)
    refpt = samples[idx]
    # Try to pick a new point relative to the reference point.
    pt = get_point(k, refpt)
    if pt:
        # Point pt is valid: add it to the samples list and mark it as active
        samples.append(pt)
        nsamples += 1
        active.append(len(samples)-1)
        cells[get_cell_coords(pt)] = len(samples) - 1
    else:
        # We had to give up looking for valid points near refpt, so remove it
        # from the list of "active" points.
        active.remove(idx)

plt.scatter(*zip(*samples), color='r', alpha=0.6, lw=0)
plt.xlim(0, width)
plt.ylim(0, height)
plt.axis('off')
plt.show()

In [ ]:
def noise_shift(x, shft):
    return x + shft

def noise_random(x, std):
    noise = np.random.normal(0, std, (len(x), 2))
    x = x + noise
    return(x)

In [ ]:
def acc_nearest_neighbours(x_drone, x_ground):
    n = len(x_drone)
    x_drone_nn = np.hstack((x_drone, np.zeros((n, 1), dtype=np.int32)))
    x_ground_nn = np.hstack((x_ground, np.zeros((n, 1), dtype=np.int32)+1))
    idx = index_nn(x_ground_nn, x_drone_nn)
    acc = accuracy_score(range(n), idx)
    return acc

def acc_optimal_transport_non_greedy(x_drone, x_ground):
    n = len(x_drone)
    G = OT_sinkhorn(x_drone, x_ground, lambd = 0.01)
    idx = np.argmax(G, axis = 0)
    acc = accuracy_score(range(n), idx)
    return acc

def acc_optimal_transport_greedy(x_drone, x_ground):
    n = len(x_drone)
    G = OT_sinkhorn(x_drone, x_ground, lambd = 0.01)
    cost = -G*10e4
    row_ind, col_ind = linear_sum_assignment(cost)
    acc = accuracy_score(range(n), col_ind)
    return acc

def acc_gromov_wasserstein(x_drone, x_ground):
    n = len(x_drone)
    G = gromov_wasserstein(x_drone, x_ground)
    idx = np.argmax(G, axis = 0)
    acc = accuracy_score(range(n), idx)
    return acc

In [ ]:
def evaluation_noise(samples, shifts, sigmas):
    
    res = pd.DataFrame()

    n = len(samples)
    
    for s in range(len(shifts)):
        for p in range(len(sigmas)):
            std = sigmas[p]
            shft = shifts[s]
            
            x_ground = noise_random(samples, std)
            x_drone = noise_shift(samples, shft)
    
            # Nearest Neighbours
            acc = acc_nearest_neighbours(x_drone, x_ground)
            new_row = {'method':'NN', 'std':std, 'shift':shft, 'accuracy':acc}
            #append row to the dataframe
            res = res.append(new_row, ignore_index=True)

            
            # OT ground to drone
            acc = acc_optimal_transport_non_greedy(x_drone, x_ground)
            new_row = {'method':'OT non greedy', 'std':std, 'shift':shft, 'accuracy':acc}
            #append row to the dataframe
            res = res.append(new_row, ignore_index=True)

            # OT greedy
            acc = acc_optimal_transport_greedy(x_drone, x_ground)
            new_row = {'method':'OT greedy', 'std':std, 'shift':shft, 'accuracy':acc}
            #append row to the dataframe
            res = res.append(new_row, ignore_index=True)


            # Gromov-Wasserstein
            acc_gromov_wasserstein(x_drone, x_ground)
            new_row = {'method':'GW', 'std':std, 'shift':shft, 'accuracy':acc}
            #append row to the dataframe
            res = res.append(new_row, ignore_index=True)
        
        
    return(res)

In [ ]:
#shifts = np.array([0, 0.5, 1, 1.5, 2])
#sigmas = np.array([0, 0.5, 1, 1.5, 2])

shifts = np.linspace(0, 2, 11)
sigmas = np.linspace(0, 2, 11)

res = evaluation_noise(samples, shifts, sigmas)

In [ ]:
res.to_csv('results/results_gps_noise_2D_nn_ot_gw.csv') # If 200 points simulated in the synthetic dataset
#res.to_csv('results/results_gps_noise_2D_nn_ot_gw_small.csv') # If 50 points simulated in the synthetic dataset

In [ ]:
# If 200 points simulated in the synthetic dataset
res = pd.read_csv('results/results_gps_noise_2D_nn_ot_gw.csv', index_col = 0)
res_gmn = pd.read_csv('results/results_gps_noise_2D_gmn.csv', index_col = 0)
res = pd.concat([res, res_gmn])

# If 50 points simulated in the synthetic dataset
res = pd.read_csv('results/results_gps_noise_2D_nn_ot_gw_small.csv', index_col = 0)
res_gmn = pd.read_csv('results/results_gps_noise_2D_gmn_small.csv', index_col = 0)
res = pd.concat([res, res_gmn])

In [ ]:
methods = ['NN', 'GMN', 'OT greedy', 'OT non greedy', 'GW']

res['shift'] = round(res['shift'], 2)
res['std'] = round(res['std'], 2)

fig = plt.figure(figsize=(20, 10))
gs = gridspec.GridSpec(2, 3, figure=fig)
gs.update(wspace = 0.1, hspace = 0.3)

for i in range (len(methods)):
    df_sub = res[res.method==methods[i]]
    result = df_sub.pivot(index='shift', columns='std', values='accuracy')
    ax = fig.add_subplot(gs[i//3, i%3])
    ax.set_title(methods[i], fontsize = 14)
    ax.set_ylabel('tree detection accuracy')
    sns.heatmap(result, cmap='viridis', ax = ax, vmin=0, vmax=1)
    ax.set(xlabel="Individual noise in ground data - sigma (m)", ylabel = "General noise in drone data - shift (m)")

fig.suptitle('Evaluation of Mapping Methods when Increasing Noise in GPS position of Ground and Drone Measurements\n (Synthetic Dataset with 200 trees)', fontsize = 16)
plt.show()

## Random Noise in DeepForest & GPS Position of ground data

We are interested to change deepforest ability to detect trees on drone images. We modify the threshold score of the algorithm. From [Ben. G. Weinstein et al., 2019], the score_threshold is defined as the minimum probability score to be included in final boxes, ranging from 0 to 1. We increase the threshold score from 0 to 0.7 to refine the tree detection.

We work on a synthetic dataset based on the drone image of the site BART from NEON dataset. More precisely, we focus on the tile number 50 (BART_050). There are 52 tree instances (ground measurements).

In [ ]:
dir = os.getcwd()
img_path = 'NEON/evaluation/BART_050.tif'

In [ ]:
test_model = deepforest.deepforest()
test_model.use_release()

In [ ]:
thresholds = np.linspace(0, 1, 11)
for k in thresholds:
    boxes = get_annotations(os.path.join(dir,img_path), test_model, score_threshold = k)
    boxes.to_csv('NEON/annotations/annot_threshold_{}.csv'.format(k))

In [ ]:
src = rasterio.open(img_path)
raster = src.read()
image = reshape_as_image(raster)

# Plot all points
fig, ax = plt.subplots(1, figsize = (15,15))
im = mpimg.imread(img_path)

#ax = sns.scatterplot(x="x", y="y", data=df_img, s = 150, hue="label", edgecolor='black',linewidth=1, palette = palette)
ax.imshow(im)
#plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

im = Image.fromarray(im)
im.save('NEON/images/BART_050.png')

In [ ]:
# Plot the 52 tree instances predicted by the released version of DeepForest as the truth.
im = cv2.imread(img_path)
df_drone = pd.read_csv('NEON/annotations/final_annotations.csv', index_col = 0)
df_img = df_drone[df_drone.img_path == 'BART_050.tif']

df_img = df_img.reset_index(drop=True)
df_img['tag'] = df_img.index

print(len(df_img))

In [ ]:
# Visualise true bounding boxes
true_boxes = df_img[['xmin', 'ymin', 'xmax', 'ymax']].to_numpy()
true_annotations = box_to_annotation(true_boxes)
im = draw_annotations(im, true_annotations, color=(255, 0, 0), label_to_name=None, show_caption = True, cv2_authorized = False, thickness = 2)
plt.show()

In [ ]:
def noise_ground(dfg, std):
    n = len(dfg)
    dfg_noisy = dfg.copy()
    noise = np.random.normal(0, std, (n, 2))
    dfg_noisy[['x', 'y']] = dfg[['x', 'y']] + noise
    return dfg_noisy

def get_tag(new_boxes, true_boxes):
    X = new_boxes[['xmin', 'ymin', 'xmax', 'ymax']].to_numpy()
    gt = true_boxes[['xmin', 'ymin', 'xmax', 'ymax']].to_numpy()
    p = 0
    
    for i in range(len(X)):
        iou = []
        for j in range(len(gt)):
            iou.append(bb_intersection_over_union(X[i],gt[j]))
        if np.max(iou)>0.5:
            p += 1
            new_boxes.loc[i,'tag'] = true_boxes.loc[np.argmax(iou),'tag']
        else:
            new_boxes.loc[i,'tag'] = -1
    print('There are {} true boxes'.format(p))
    return(new_boxes)

In [ ]:
thresholds = np.linspace(0, 1, 11)
sigmas = np.linspace(0, 2, 11)*10

In [ ]:
dir = os.getcwd()
img_path = 'NEON/images/BART_050.tif'

res = pd.DataFrame()

df_ground = df_img[['x', 'y', 'tag']]
df_drone = df_img[['xmin', 'ymin', 'xmax', 'ymax', 'x', 'y', 'tag']]
n = len(df_ground)

for k in thresholds:
    boxes = get_annotations(os.path.join(dir,img_path), test_model, score_threshold = k)
    
    # Visualisation of the predicted bounding boxes
    im = cv2.imread(img_path)
    annotations = box_to_annotation(boxes[['xmin', 'ymin', 'xmax', 'ymax']].to_numpy())
    im = draw_annotations(im, annotations, color=(255, 0, 0), label_to_name=None, show_caption = True, cv2_authorized = False, thickness = 2)
    plt.show()
    
    boxes = get_tag(boxes, df_drone)
    boxes[['x', 'y']] = boxes.apply(lambda x: [(x.xmin+x.xmax)/2, (x.ymin+x.ymax)/2], axis=1, result_type="expand")

    print(boxes.tag.to_numpy())
    
    for s in range(len(sigmas)):
        std = sigmas[s]
        dfd_0 = boxes
        dfg_0 = noise_ground(df_ground, std)
        m = len(dfd_0)
        
        X_drone = dfd_0[['x', 'y']].to_numpy()
        X_ground = dfg_0[['x', 'y']].to_numpy()
        
        X_drone_true = df_drone[['x', 'y']].to_numpy()
        
        plt.figure(figsize=(15,15))
        plt.imshow(im.astype(int))
        for i in range(len(X_drone)):
            plt.scatter(x=X_drone[i][0], y=X_drone[i][1], c='r', s=40)
        for i in range(len(X_drone_true)):
            plt.scatter(x=X_drone_true[i][0], y=X_drone_true[i][1], c='k', s=100)
        for i in range(len(X_ground)):
            plt.scatter(x=X_ground[i][0], y=X_ground[i][1], c='b', s=40)
        plt.show()


        # Nearest Neighbours
        dfd = dfd_0.copy()
        dfg = dfg_0.copy()
        X_drone_nn = np.hstack((X_drone, np.zeros((m, 1), dtype=np.int32)))
        X_ground_nn = np.hstack((X_ground, np.zeros((n, 1), dtype=np.int32)+1))
        idx = index_nn(X_ground_nn, X_drone_nn)
        dfg['idx'] = idx
        dfd['idx']= dfd.index
        final = pd.merge(dfd, dfg, on='idx', how = 'inner', suffixes=('_d', '_g'))
        acc = len(final[final.tag_d == final.tag_g])/n
        new_row = {'method':'NN', 'std':std, 'deepforest_threshold':k, 'accuracy':acc}
        #append row to the dataframe
        res = res.append(new_row, ignore_index=True)


        # OT ground to drone
        dfd = dfd_0.copy()
        dfg = dfg_0.copy()
        G = OT_sinkhorn(X_drone, X_ground, lambd = 0.01)
        idx = np.argmax(G, axis = 0)
        dfg['idx'] = idx
        dfd['idx']= dfd.index
        final = pd.merge(dfd, dfg, on='idx', how = 'inner', suffixes=('_d', '_g'))
        acc = len(final[final.tag_d == final.tag_g])/n
        new_row = {'method':'OT non greedy', 'std':std, 'deepforest_threshold':k, 'accuracy':acc}
        #append row to the dataframe
        res = res.append(new_row, ignore_index=True)

        # OT greedy
        dfd = dfd_0.copy()
        dfg = dfg_0.copy()
        G = OT_sinkhorn(X_drone, X_ground, lambd = 0.01)
        cost = -G*10e4
        row_ind, col_ind = linear_sum_assignment(cost)
        df_left = dfd.loc[row_ind]
        df_left['idx']= col_ind
        dfg['idx'] = dfg.index
        final = pd.merge(df_left, dfg, on='idx', how = 'inner', suffixes=('_d', '_g')) 
        acc = len(final[final.tag_d == final.tag_g])/n
        new_row = {'method':'OT greedy', 'std':std, 'deepforest_threshold':k, 'accuracy':acc}
        #append row to the dataframe
        res = res.append(new_row, ignore_index=True)


        # Gromov-Wasserstein
        G = gromov_wasserstein(X_drone, X_ground)
        idx = np.argmax(G, axis = 0)
        dfg['idx'] = idx
        dfd['idx']= dfd.index
        final = pd.merge(dfd, dfg, on='idx', how = 'inner', suffixes=('_d', '_g'))
        print(final)
        acc = len(final[final.tag_d == final.tag_g])/n
        new_row = {'method':'GW', 'std':std, 'deepforest_threshold':k, 'accuracy':acc}
        #append row to the dataframe
        res = res.append(new_row, ignore_index=True)

        print(res)

    

In [ ]:
res.to_csv('NEON/evaluation/results_gps_deepforest_noise_nn_ot_gw.csv')

In [ ]:
res = pd.read_csv('NEON/evaluation/results_gps_deepforest_noise_nn_ot_gw.csv', index_col = 0)
res_gmn = pd.read_csv('NEON/evaluation/results_gps_deepforest_noise_gmn.csv', index_col = 0)
res = pd.concat([res, res_gmn])

In [ ]:
methods = ['NN', 'GMN', 'OT greedy', 'OT non greedy', 'GW']

res['deepforest_threshold'] = round(res['deepforest_threshold'], 2)
res['std'] = round(res['std'], 2)

import matplotlib.gridspec as gridspec
fig = plt.figure(figsize=(20, 10))
gs = gridspec.GridSpec(2, 3, figure=fig)
gs.update(wspace = 0.1, hspace = 0.3)

for i in range (len(methods)):
    df_sub = res[res.method==methods[i]]
    result = df_sub.pivot(index='deepforest_threshold', columns='std', values='accuracy')
    ax = fig.add_subplot(gs[i//3, i%3])
    ax.set_title(methods[i], fontsize = 14)
    sns.heatmap(result, cmap='viridis', ax = ax, vmin=0, vmax=1)
    ax.set(xlabel="Individual noise in ground data - sigma (m)", ylabel = "Deepforest threshold to keep boxes")

fig.suptitle('Evaluation of mapping methods when increasing noise in GPS position of ground measurements and tree detection threshold of DeepForest\n (Synthetic Dataset on BART_050 (NEON) with 50 ground measuremnts)', fontsize = 16)
plt.show()